In [1]:
#Network x tools.
import networkx as nx
import networkx.algorithms as algo

import time
import pandas as pd

#spotify tools.
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

### Authenticating with Spotify API

In [2]:
client_id = '85b32a4fe99f44db9d9665c3281bd8a5'
client_secret = '8598677237e74986ac35810334a418d3'
username = 'ks65h2anf54k24x4n4jvfx2hy'

# set up Spotify API credentials
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Gathering data

In [3]:
# First set of 50 songs, search query and parameters
query = 'year:2022'
limit = 50
offset = 0
# Spotify API call
results = sp.search(q=query, type='track', limit=limit, offset=offset)

# Second set of 50 songs, search query and parameters
query2 = 'year:2022'
limit = 50
offset = 50
# Spotify API call
results2 = sp.search(q=query, type='track', limit=limit, offset=offset)

In [4]:
# Combine the two sets of results into a single dictionary
all_results = {'tracks': {'items': []}}
all_results['tracks']['items'].extend(results['tracks']['items'])
all_results['tracks']['items'].extend(results2['tracks']['items'])

In [11]:
# create an empty DataFrame with columns 'track_id', 'artist_name', 'tempo', 'track_name'
song_info_df = pd.DataFrame(columns=['track_id', 'track_name', 'artist_name', 'tempo', 'popularity'])

# iterate over the tracks and add the track id, artist name, and track name to the DataFrame
for track in all_results['tracks']['items']:
    track_id = track['id']
    artist_names = [artist['name'] for artist in track['album']['artists']]
    artist_name = ', '.join(artist_names)
    track_name = track['name']
    tempo = None
    popularity = track['popularity']
    song_info_df.loc[len(song_info_df)] = [track_id, track_name, artist_name, tempo, popularity]
    
song_info_df


,track_id,track_name,artist_name,tempo,popularity
0,1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,None,95
1,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,None,97
2,4FyesJzVpA39hbYvcseO2d,Just Wanna Rock,Lil Uzi Vert,None,91
3,0vjeOZ3Ft5jvAi9SBFJm1j,Superhero (Heroes & Villains) [with Future & C...,Metro Boomin,None,91
4,1bDbXMyjaUIooNwFE9wn0N,Rich Flex,"Drake, 21 Savage",None,91
...,...,...,...,...,...
95,2tTmW7RDtMQtBk7m2rYeSw,"Quevedo: Bzrp Music Sessions, Vol. 52","Bizarrap, Quevedo",None,93
96,37F7E7BKEw2E4O2L7u0IEp,Limbo,Freddie Dredd,None,85
97,4OmfWzukSVD140NiAIEjem,Static,Steve Lacy,None,81
98,0IKeDy5bT9G0bA7ZixRT4A,Bebe Dame,Fuerza Regida,None,85


In [14]:
# Add tempo column to dataframe
def get_tempo(track_id):
    features = sp.audio_features(track_id)
    if features is not None and len(features) > 0 and features[0] is not None:
        return features[0]['tempo']
    else:
        return None

song_info_df['tempo'] = song_info_df['track_id'].apply(get_tempo)
song_info_df

,track_id,track_name,artist_name,tempo,popularity
0,1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,88.980,95
1,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,97.950,97
2,4FyesJzVpA39hbYvcseO2d,Just Wanna Rock,Lil Uzi Vert,150.187,91
3,0vjeOZ3Ft5jvAi9SBFJm1j,Superhero (Heroes & Villains) [with Future & C...,Metro Boomin,116.622,91
4,1bDbXMyjaUIooNwFE9wn0N,Rich Flex,"Drake, 21 Savage",153.150,91
...,...,...,...,...,...
95,2tTmW7RDtMQtBk7m2rYeSw,"Quevedo: Bzrp Music Sessions, Vol. 52","Bizarrap, Quevedo",128.033,93
96,37F7E7BKEw2E4O2L7u0IEp,Limbo,Freddie Dredd,74.987,85
97,4OmfWzukSVD140NiAIEjem,Static,Steve Lacy,79.001,81
98,0IKeDy5bT9G0bA7ZixRT4A,Bebe Dame,Fuerza Regida,157.643,85


### Creating network

In [9]:
# Create empty graph
G = nx.Graph()

# Add nodes to graph for each track
for _, row in song_info_df.iterrows():
    G.add_node(row['track_id'])

# Add edges to graph for each pair of tracks that share an artist, tempo, or additional artist
for _, row1 in song_info_df.iterrows():
    for _, row2 in song_info_df.iterrows():
        if row1['track_id'] != row2['track_id']:
            if row1['artist_name'] == row2['artist_name'] or abs(row1['tempo'] - row2['tempo']) <= 5:
                G.add_edge(row1['track_id'], row2['track_id'])
            else:
                # Check if additional artists are the same
                if 'featuring' in row1['track_name'] and 'featuring' in row2['track_name']:
                    artists1 = [a.strip() for a in row1['track_name'].split('featuring')[1].split(',')]
                    artists2 = [a.strip() for a in row2['track_name'].split('featuring')[1].split(',')]
                    if set(artists1) & set(artists2):
                        G.add_edge(row1['track_id'], row2['track_id'])

print(f"Number of nodes in graph: {G.number_of_nodes()}")
print(f"Number of edges in graph: {G.number_of_edges()}")

Number of nodes in graph: 50
Number of edges in graph: 208
